In [1]:
import diskcache as dc
import os
import xarray
import s3fs

ModuleNotFoundError: No module named 's3fs'

In [3]:
cache = dc.Cache(directory='./diskcache')
@cache.memoize(tag='diskcache_xarray_open_dataset')
def xarray_open_dataset(src_path: str) -> xarray.Dataset:
    """Open dataset."""
    xr_open_args: Dict[str, Any] = {
        "decode_coords": "all",
        "decode_times": False,
        "chunks": None,
        "engine": "h5netcdf"
    }
    fs = s3fs.S3FileSystem()
    file_handler = fs.open(src_path)
    return xarray.open_dataset(file_handler, **xr_open_args)

AttributeError: module 'diskcache' has no attribute 'Cache'

In [3]:
%%time
src_path = 's3://nasa-veda-scratch/cmip6-staging/prXin/ACCESS-CM2/prXin-ACCESS-CM2-ssp126.nc'
ds1 = xarray_open_dataset(src_path)

/Users/aimeebarciauskas/miniconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


CPU times: user 988 ms, sys: 211 ms, total: 1.2 s
Wall time: 1.95 s


In [4]:
%%time
src_path = 's3://veda-data-store-staging/rechunking/prXin-ACCESS-CM2-ssp126_compressed.nc'
ds2 = xarray_open_dataset(src_path)

CPU times: user 1.1 s, sys: 261 ms, total: 1.36 s
Wall time: 24.6 s


In [5]:
%%time
loaded = ds1.load()
loaded

CPU times: user 21.9 s, sys: 7.86 s, total: 29.8 s
Wall time: 6min 2s


<xarray.Dataset>
Dimensions:  (time: 86, lat: 600, lon: 1440)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) int64 0 366 731 1096 1461 ... 29586 29951 30316 30681 31046
Data variables:
    prXin_1  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_2  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_3  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_4  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    cmip6_source_id:  ACCESS-CM2
    scenario:         ssp126

In [6]:
%%time
ds2.load()

CPU times: user 11.6 s, sys: 1.73 s, total: 13.3 s
Wall time: 16.1 s


<xarray.Dataset>
Dimensions:  (time: 86, lat: 600, lon: 1440)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) int64 0 366 731 1096 1461 ... 29586 29951 30316 30681 31046
Data variables:
    prXin_1  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_2  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_3  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_4  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    cmip6_source_id:  ACCESS-CM2
    scenario:         ssp126

In [7]:
display_variable = 'prXin_1'
ds_rechunked = ds1.chunk(chunks={'time': 1, 'lat': 600, 'lon': 1440})
ds_rechunked[display_variable]

<xarray.DataArray 'prXin_1' (time: 86, lat: 600, lon: 1440)>
dask.array<xarray-prXin_1, shape=(86, 600, 1440), dtype=float64, chunksize=(1, 600, 1440), chunktype=numpy.ndarray>
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) int64 0 366 731 1096 1461 ... 29586 29951 30316 30681 31046
Attributes:
    description:  Annual number of days with total precipitation greater than...
    units:        Days

In [8]:
%%time
target_file_name = "prXin-ACCESS-CM2-ssp126_small_chunks_compressed.nc"
comp = dict(zlib=True, complevel=5)
encoding = {var: comp for var in ds1.data_vars}
loaded.to_netcdf(target_file_name, encoding=encoding)

CPU times: user 11.5 s, sys: 95.7 ms, total: 11.6 s
Wall time: 11.7 s


In [9]:
print(f"Compressed netCDF size {os.path.getsize(target_file_name)/1000000} MB")

Compressed netCDF size 23.582471 MB


In [10]:
fs = s3fs.S3FileSystem()
new_target = f"s3://nasa-veda-scratch/cmip6-staging/{target_file_name}"
fs.put(target_file_name, new_target)

[None]

In [11]:
%%time
ds3 = xarray_open_dataset(new_target)

CPU times: user 749 ms, sys: 213 ms, total: 962 ms
Wall time: 18.6 s


In [12]:
%%time
ds3.load()

CPU times: user 11.9 s, sys: 1.76 s, total: 13.6 s
Wall time: 17.8 s


<xarray.Dataset>
Dimensions:  (time: 86, lat: 600, lon: 1440)
Coordinates:
  * lat      (lat) float64 -59.88 -59.62 -59.38 -59.12 ... 89.38 89.62 89.88
  * lon      (lon) float64 -179.9 -179.6 -179.4 -179.1 ... 179.4 179.6 179.9
  * time     (time) int64 0 366 731 1096 1461 ... 29586 29951 30316 30681 31046
Data variables:
    prXin_1  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_2  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_3  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
    prXin_4  (time, lat, lon) float64 nan nan nan nan nan ... nan nan nan nan
Attributes:
    cmip6_source_id:  ACCESS-CM2
    scenario:         ssp126